In [1]:
# Import dependencies
import os

os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

from src.utils.gpu_config import configure_gpu, optimize_tensorflow

configure_gpu()
optimize_tensorflow()

2025-05-21 05:51:39.893851: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747799499.994756    4041 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747799500.024001    4041 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747799500.263228    4041 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747799500.263237    4041 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747799500.263238    4041 computation_placer.cc:177] computation placer alr


GPU configuration successful!
Device: /physical_device:GPU:0
Memory limit: 18.0 GB

TensorFlow optimizations status:
- CPU Threads: 16 intra-op, 9 inter-op (32 cores total)
- XLA JIT: enabled (autoclustering)
- Synchronous Execution: enabled

GPU Device Details:
- Device: GPU /physical_device:GPU:0
- Memory limit: 18.0 GB


I0000 00:00:1747799503.807561    4041 gpu_process_state.cc:208] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1747799503.808403    4041 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 19327352832 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


In [2]:
# Import Tensorflow Keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryFocalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Import local modules
from src.utils.consts import TF_RECORD_DATASET, MODELS_PATH, TF_BUFFER_SIZE, TF_SHUFFLE_SIZE, TF_BATCH_SIZE, TF_MAX_EPOCHS, NO_FINDING_CLASS_IDX
from src.model.tensorflow_utils import load_dataset, optimize_dataset, count_dataset_size, get_num_classes, get_metrics
from src.model.tensorflow_utils import setup_logger, setup_training_logger, setup_metrics_monitor, setup_early_stopping, filter_no_finding_class
from src.model.tensorflow_utils import start_or_resume_training, apply_augmentation_to_dataset, calculate_class_weights, oversample_minority_classes
from src.model.densnet.tensorflow_dense_net_121 import build_densenet121
from src.model.loss import NoFindingBinaryCrossentropy

# Input Data
initial_epoch   = 30
resume_training = True
checkpoint_path = '/home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV2_6/checkpoints/cp-0029.keras'
set_name        = "DenseNetV2"
model_name      = f"{set_name}_6"
num_classes     = get_num_classes()

train_ds = load_dataset(f"{TF_RECORD_DATASET}/train.tfrecord", TF_BUFFER_SIZE)
val_ds   = load_dataset(f"{TF_RECORD_DATASET}/val.tfrecord", TF_BUFFER_SIZE)

class_weights = calculate_class_weights(train_ds, num_classes)
train_ds      = oversample_minority_classes(train_ds, class_weights)
class_weights = calculate_class_weights(train_ds, num_classes)

In [3]:
steps_per_epoch  = int(count_dataset_size(train_ds, None) / TF_BATCH_SIZE)
validation_steps = int(count_dataset_size(val_ds, None) / TF_BATCH_SIZE)

# Disable Advanced Augmentations
probability = {
    "intensity_scaling": 0,
    "adaptive_histogram": 0,
    "zoom": 0,
    "cutout": 0
}

train_ds = train_ds.shuffle(TF_SHUFFLE_SIZE, reshuffle_each_iteration=True)
train_ds = apply_augmentation_to_dataset(train_ds, probability)
train_ds = optimize_dataset(train_ds, TF_BATCH_SIZE)

val_ds   = optimize_dataset(val_ds, TF_BATCH_SIZE)

Counting samples: 111784 samples [00:42, 2641.59 samples/s] 
Counting samples: 15431 samples [00:14, 1039.99 samples/s]


In [4]:
# Setup Model Deps
logger            = setup_logger()
training_logger   = setup_training_logger(logger, TF_BATCH_SIZE, 100)
metrics_monitor   = setup_metrics_monitor(MODELS_PATH, model_name, logger, resume_training=resume_training, initial_epoch=initial_epoch)
metrics           = get_metrics()

# Setup compile arguments
loss           = NoFindingBinaryCrossentropy(no_finding_idx=NO_FINDING_CLASS_IDX, from_logits=False)
reduce_lr      = ReduceLROnPlateau(monitor="val_f1_score", factor=0.5,  patience=3, min_lr=1e-6, mode="max", verbose=1)
early_stopping = setup_early_stopping()

epoch_mode           = 'cp-{epoch:04d}'
save_checkpoint_path = f"{MODELS_PATH}/{model_name}/checkpoints/{epoch_mode}.keras"
checkpoint           = ModelCheckpoint(save_checkpoint_path, monitor="val_f1_score", save_best_only=False, mode="max", save_weights_only=False)

model_path      = f"{MODELS_PATH}/{model_name}.keras"
best_checkpoint = ModelCheckpoint(model_path, monitor="val_f1_score", save_best_only=True, mode="max", save_weights_only=False)

2025-05-21 05:54:29 - INFO - Resuming from existing metrics file: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV2_6/train_metrics.csv
2025-05-21 05:54:29 - INFO - Resuming from existing validation metrics file: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV2_6/val_metrics.csv
2025-05-21 05:54:29 - INFO - Cleaned training metrics file, kept 25317 records before epoch 30
2025-05-21 05:54:29 - INFO - Cleaned validation metrics file, kept 29 records before epoch 30
2025-05-21 05:54:29 - INFO - Found 25317 existing training records
2025-05-21 05:54:29 - INFO - Found 29 existing validation records


In [5]:
# Model Training
model          = build_densenet121(num_classes)
optimizer      = Adam(learning_rate=1e-4)
compile_kwargs = {'optimizer': optimizer, 'loss': loss, 'metrics': metrics}

history, model = start_or_resume_training(
    model, 
    compile_kwargs, 
    train_ds, 
    val_ds, 
    TF_MAX_EPOCHS,
    steps_per_epoch, 
    validation_steps, 
    class_weights=class_weights,
    callbacks=[checkpoint, best_checkpoint, reduce_lr, training_logger, metrics_monitor, early_stopping], 
    checkpoint_path=checkpoint_path,
    initial_epoch=initial_epoch,
    output_dir=MODELS_PATH,
    model_name=model_name,
    logger=logger
)

2025-05-21 05:54:31 - INFO - Resuming training from checkpoint: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV2_6/checkpoints/cp-0029.keras
2025-05-21 05:54:33 - INFO - Resuming from existing metrics file: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV2_6/train_metrics.csv
2025-05-21 05:54:33 - INFO - Resuming from existing validation metrics file: /home/piotr/Pulpit/codebook/studies/bachelor-thesis/models/DenseNetV2_6/val_metrics.csv
2025-05-21 05:54:33 - INFO - Cleaned training metrics file, kept 25317 records before epoch 30
2025-05-21 05:54:33 - INFO - Cleaned validation metrics file, kept 29 records before epoch 30
2025-05-21 05:54:33 - INFO - Found 25317 existing training records
2025-05-21 05:54:33 - INFO - Found 29 existing validation records
2025-05-21 05:54:33 - INFO - 
=== Training Started ===

2025-05-21 05:54:33 - INFO - Batch Size: 128
2025-05-21 05:54:33 - INFO - Optimizer: Adam
2025-05-21 05:54:33 - INFO - 

2025-05-21 05:54:3

Epoch 30/100


I0000 00:00:1747799697.399435    4097 service.cc:152] XLA service 0x76d0600469a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747799697.399483    4097 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3090 Ti, Compute Capability 8.6
I0000 00:00:1747799704.502547    4097 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1747799769.059259    4097 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


873/873 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step - accuracy: 0.9596 - auc: 0.9478 - f1_score: 0.7552 - loss: 0.2379 - precision: 0.8624 - recall: 0.6838  

2025-05-21 06:00:20 - INFO - 
=== Epoch 30 Summary ===
2025-05-21 06:00:20 - INFO - Time: 346.72s
2025-05-21 06:00:20 - INFO - Training   - accuracy: 0.9611 - auc: 0.9515 - f1_score: 0.7648 - loss: 0.2355 - precision: 0.8693 - recall: 0.6937 - learning_rate: 0.0000
2025-05-21 06:00:20 - INFO - Validation - accuracy: 0.9186 - auc: 0.7108 - f1_score: 0.4212 - loss: 0.5137 - precision: 0.5290 - recall: 0.3992
2025-05-21 06:00:20 - INFO - ==================================================



...

Epoch 47/100
873/873 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step - accuracy: 0.9681 - auc: 0.9632 - f1_score: 0.8122 - loss: 0.2256 - precision: 0.8964 - recall: 0.7507  

2025-05-21 07:10:58 - INFO - 
=== Epoch 47 Summary ===
2025-05-21 07:10:58 - INFO - Time: 249.81s
2025-05-21 07:10:58 - INFO - Training   - accuracy: 0.9686 - auc: 0.9647 - f1_score: 0.8156 - loss: 0.2239 - precision: 0.8982 - recall: 0.7544 - learning_rate: 0.0000
2025-05-21 07:10:58 - INFO - Validation - accuracy: 0.9158 - auc: 0.7039 - f1_score: 0.4178 - loss: 0.5299 - precision: 0.5073 - recall: 0.3940
2025-05-21 07:10:58 - INFO - ==================================================



873/873 ━━━━━━━━━━━━━━━━━━━━ 250s 286ms/step - accuracy: 0.9681 - auc: 0.9632 - f1_score: 0.8122 - loss: 0.2256 - precision: 0.8964 - recall: 0.7507 - val_accuracy: 0.9158 - val_auc: 0.7039 - val_f1_score: 0.4178 - val_loss: 0.5299 - val_precision: 0.5073 - val_recall: 0.3940 - learning_rate: 1.0000e-06


2025-05-21 07:10:58 - INFO - 
=== Training Completed! ===

2025-05-21 07:10:58 - INFO - Final Metrics: accuracy: 0.9686 - auc: 0.9647 - f1_score: 0.8156 - loss: 0.2239 - precision: 0.8982 - recall: 0.7544 - val_accuracy: 0.9158 - val_auc: 0.7039 - val_f1_score: 0.4178 - val_loss: 0.5299 - val_precision: 0.5073 - val_recall: 0.3940



Epoch 47: early stopping
Restoring model weights from the end of the best epoch: 32.
